-   [Using Bioconductor for High Throughput
    Assays](#using-bioconductor-for-high-throughput-assays)
    -   [Flow Cytometry](#flow-cytometry)
    -   [Cell-based Assays](#cell-based-assays)
    -   [High-throughput qPCR Assays](#high-throughput-qpcr-assays)
    -   [Mass Spectrometry and Proteomics
        data](#mass-spectrometry-and-proteomics-data)
    -   [Imaging Based Assays](#imaging-based-assays)

Using Bioconductor for High Throughput Assays
=============================================

Bioconductor includes packages for analysis of diverse areas of high-throughput assays such as flow cytometry, quantitative real-time PCR, mass spectrometry, proteomics and other cell-based data.

-   [Sample Workflow](#sample-workflow)
-   [Installation and Use](#install-and-use)
-   [Exploring Package Content](#exploring-package-content)
-   [Diverse Assays Resources](#diverse-assays-resources)

<h2 id="sample-workflow">
Sample Workflow
</h2>
The following psuedo-code illustrates a typical R / Bioconductor session. It makes use of the flow cytometry packages to load, transform and visualize the flow data and gate certain populations in the dataset.

The workflow loads the `flowCore`, `flowStats` and `flowViz` packages and its dependencies. It loads the ITN data with 15 samples, each of which includes, in addition to FSC and SSC, 5 fluorescence channels: CD3, CD4, CD8, CD69 and HLADR.

In [1]:
## Load packages
library(flowCore)
library(flowStats)
library(flowViz) # for flow data visualization

## Load data
data(ITN)
ITN

Loading required package: fda
Loading required package: splines
Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:flowCore’:

    %&%


Attaching package: ‘fda’

The following object is masked from ‘package:graphics’:

    matplot

Loading required package: mvoutlier
Loading required package: sgeostat
sROC 0.1-2 loaded
Loading required package: cluster
Loading required package: flowWorkspace
Loading required package: flowViz
Loading required package: lattice
Loading required package: ncdfFlow
Loading required package: RcppArmadillo
Loading required package: BH
Loading required package: gridExtra


A flowSet with 15 experiments.

An object of class 'AnnotatedDataFrame'
  rowNames: sample01 sample02 ... sample15 (15 total)
  varLabels: GroupID SiteCode ... name (7 total)
  varMetadata: labelDescription

  column names:
  FSC SSC CD8 CD69 CD4 CD3 HLADr Time

First, we need to transform all the fluorescence channels. Using a `workFlow` object can help to keep track of our progress.

In [2]:
## Create a workflow instance and transform data using asinh
wf <- workFlow(ITN)
asinh <- arcsinhTransform()
tl <- transformList(colnames(ITN)[3:7], asinh, 
                      transformationId = "asinh")
add(wf, tl)

Warning message:
: 'workFlow' is deprecated.
Use 'flowWorkspace::GatingSet' instead.
See help("Deprecated")

Next we use the `lymphGate` function to find the T-cells in the CD3/SSC projection.

``` r
lg <- lymphGate(Data(wf[["asinh"]]), channels=c("SSC", "CD3"),
         preselection="CD4", filterId="TCells", eval=FALSE,
         scale=2.5)
add(wf, lg$n2gate, parent="asinh")
print(xyplot(SSC ~ CD3| PatientID, wf[["TCells+"]],
             par.settings=list(gate=list(col="red", 
             fill="red", alpha=0.3))))
```

A typical workflow for flow cytometry data analysis in Bioconductor flow packages include data transformation, normalization, filtering, manual gating, semi-automatic gating and automatic clustering if desired. Details can be found in [flowWorkFlow.pdf](flowWorkFlow.pdf) or the vignettes of the [flow cytometry packages](#diverse-assays-resources).

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="install-and-use">
Installation and Use
</h2>
Follow [installation instructions](/install/) to start using these packages. To install the `flowCore` package and all of its dependencies, evaluate the commands

In [3]:
## try http:// if https:// URLs are not supported
source("https://bioconductor.org/biocLite.R")
biocLite("flowCore")

Bioconductor version 3.2 (BiocInstaller 1.20.1), ?biocLite for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.2 (BiocInstaller 1.20.1), R 3.2.2 (2015-08-14).
Installing package(s) ‘flowCore’



The downloaded source packages are in
	‘/tmp/RtmpEm3t8d/downloaded_packages’


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


Package installation is required only once per R installation. View a full list of [available packages](/packages/release/bioc/).

To use the `flowCore` package, evaluate the command

In [4]:
library("flowCore")

This instruction is required once in each R session.

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="exploring-package-content">
Exploring Package Content
</h2>
Packages have extensive help pages, and include vignettes highlighting common use cases. The help pages and vignettes are available from within R. After loading a package, use syntax like

In [5]:
help(package="flowCore")
?read.FCS

read.FCS {flowCore},R Documentation
files,A vector of filenames
filename,Character of length 1: filename
transformation,"An character string that defines the type of transformation. Valid values are linearize (default), linearize-with-PnG-scaling, or scale. The linearize transformation applies the appropriate power transform to the data. The linearize-with-PnG-scaling transformation applies the appropriate power transform for parameters stored on log scale, and also a linear scaling transformation based on the 'gain' (FCS \$PnG keywords) for parameters stored on a linear scale. The scale transformation scales all columns to $[0,10^decades]$. defaulting to decades=0 as in the FCS4 specification. A logical can also be used: TRUE is equal to linearize and FALSE(or NULL) corresponds to no transformation. Also when the transformation keyword of the FCS header is set to ""custom"" or ""applied"", no transformation will be used."
which.lines,"Numeric vector to specify the indices of the lines to be read. If NULL all the records are read, if of length 1, a random sample of the size indicated by which.lines is read in."
alter.names,boolean indicating whether or not we should rename the columns to valid R names using make.names. The default is FALSE.
column.pattern,An optional regular expression defining parameters we should keep when loading the file. The default is NULL.
invert.pattern,"logical. By default, FALSE. If TRUE, inverts the regular expression specified in column.pattern. This is useful for indicating the channel names that we do not want to read. If column.pattern is set to NULL, this argument is ignored."
decades,"When scaling is activated, the number of decades to use for the output."
ncdf,Deprecated. Please use 'ncdfFlow' package for cdf based storage.
min.limit,"The minimum value in the data range that is allowed. Some instruments produce extreme artifactual values. The positive data range for each parameter is completely defined by the measurement range of the instrument and all larger values are set to this threshold. The lower data boundary is not that well defined, since compensation might shift some values below the original measurement range of the instrument. The default value of -111 copies the behavior of flowJo. It can be set to an arbitrary number or to NULL, in which case the original values are kept."


to obtain an overview of help on the `flowCore` package, and the `read.FCS` function, and

In [6]:
browseVignettes(package="flowCore")

starting httpd help server ... done


to view vignettes (providing a more comprehensive introduction to package functionality) in the `flowCore` package. Use

In [7]:
help.start()

If the browser launched by '/usr/bin/firefox' is already running, it is
    *not* restarted, and you must switch to its window.
Otherwise, be patient ...


to open a web page containing comprehensive help resources.

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="diverse-assays-resources">
Diverse Assays Resources
</h2>
The following provide a brief overview of packages useful for analysis of high-throughput assays. More comprehensive workflows can be found in documentation (available from [package descriptions](/packages/release/bioc/)) and in Bioconductor [publications](/help/publications/).

### Flow Cytometry

These packages use standard FCS files, including infrastructure, utilities, visualization and semi-autogating methods for the analysis of flow cytometry data.

[flowCore](/packages/release/bioc/html/flowCore.html), [flowViz](/packages/release/bioc/html/flowViz.html), [flowQ](/packages/release/bioc/html/flowQ.html), [flowStats](/packages/release/bioc/html/flowStats.html), [flowUtils](/packages/release/bioc/html/flowUtils.html), [flowFP](/packages/release/bioc/html/flowFP.html), [flowTrans](/packages/release/bioc/html/flowTrans.html),

Algorithms for clustering flow cytometry data are found in these packages:

[flowClust](/packages/release/bioc/html/flowClust.html), [flowMeans](/packages/release/bioc/html/flowMeans.html), [flowMerge](/packages/release/bioc/html/flowMerge.html), [SamSPECTRAL](/packages/release/bioc/html/SamSPECTRAL.html)

A typical workflow using the packages `flowCore`, `flowViz`, `flowQ` and `flowStats` is described in detail in [flowWorkFlow.pdf](flowWorkFlow.pdf). The data files used in the workflow can be downloaded from [here](dataFiles.tar).

### Cell-based Assays

These packages provide data structures and algorithms for cell-based high-throughput screens (HTS).

[cellHTS2](/packages/release/bioc/html/cellHTS2.html), [RNAither](/packages/release/bioc/html/RNAither.html)

This package supports the xCELLigence system which contains a series of real-time cell analyzer (RTCA).

[RTCA](/packages/release/bioc/html/RTCA.html)

### High-throughput qPCR Assays

These package provide algorithm for the analysis of cycle threshold (Ct) from quantitative real-time PCR data.

[HTqPCR](/packages/release/bioc/html/HTqPCR.html), [ddCt](/packages/release/bioc/html/ddCt.html), [qpcrNorm](/packages/release/bioc/html/qpcrNorm.html)

### Mass Spectrometry and Proteomics data

These packages provide framework for processing, visualization, and statistical analysis of mass spectral and proteomics data.

[clippda](/packages/release/bioc/html/clippda.html), [MassArray](/packages/release/bioc/html/MassArray.html), [MassSpecWavelet](/packages/release/bioc/html/MassSpecWavelet.html), [PROcess](/packages/release/bioc/html/PROcess.html), [flagme](/packages/release/bioc/html/flagme.html), [xcms](/packages/release/bioc/html/xcms.html)

### Imaging Based Assays

These packages provide infrastructure for image-based phenotyping and automation of other image-related tasks:

[EBImage](/packages/release/bioc/html/EBImage.html)

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>

In [8]:
sessionInfo()

R version 3.2.2 (2015-08-14)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 14.04.3 LTS

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] splines   stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] BiocInstaller_1.20.1      flowStats_3.28.1         
 [3] flowWorkspace_3.16.5      gridExtra_2.0.0          
 [5] ncdfFlow_2.16.0           BH_1.58.0-1              
 [7] RcppArmadillo_0.6.300.2.2 flowViz_1.34.0           
 [9] lattice_0.20-33           cluster_2.0.3            
[11] mvoutlier_2.0.6           sgeostat_1.0-26          
[13] fda_2.4.4                 Matrix_1.2-3           

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>